# OSRM (Open Source Routing Machine)

## Setup

Before proceeding with the routing examples, we ensure that the required
libraries are installed and available in the current environment.

In this notebook, we will primarily use:

- `folium` for interactive map visualization
- `pydantic` for structured data handling (if required)

The following cells verify installation and confirm the package versions
before executing the main OSRM requests.


## Table of Contents

- [Setup](#Setup)
- [Using the Service](#Using-the-Service)
  - [General Parameters](#General-Parameters)
  - [Constructing the Request](#Constructing-the-Request)
- [Visualizing the Route](#Visualizing-the-Route)
- [Route Segmentation (Leg Analysis)](#Route-Segmentation-Leg-Analysis)
- [Step-Level Inspection](#Step-Level-Inspection)
- [Intersection-Level Analysis](#Intersection-Level-Analysis)
- [Route Cost Analysis](#Route-Cost-Analysis)
- [Alternative Route Generation](#Alternative-Route-Generation)
- [Conclusion](#Conclusion)


In [ ]:
from folium import Map

In [ ]:
%pip install folium pydantic

In [ ]:
import folium
folium.__version__


The following implementation defines a coordinate model and
constructs an interactive map centered on a selected location,
with additional points plotted as markers.


In [ ]:
import folium

from folium import Map
from pydantic import BaseModel


class Point(BaseModel):
    latitude: float
    longitude: float


def get_folium_map(center_point: Point, points: list[Point], zoom_level: int = 14) -> Map:
    folium_map = folium.Map(
        location=[center_point.latitude, center_point.longitude], zoom_start=zoom_level)

    for point in points:
        folium.Marker(location=[point.latitude, point.longitude],
                      popup='Point').add_to(folium_map)

    return folium_map


point_1 = Point(latitude=53.079296, longitude=8.801693)
point_2 = Point(latitude=53.077000, longitude=8.804000)
point_3 = Point(latitude=53.075819, longitude=8.807281)

center_point = Point(latitude=53.078000, longitude=8.803000)

folium_map = get_folium_map(center_point, [point_1, point_2, point_3])
folium_map

## Using the Service

The OSRM API exposes multiple services that can be accessed
through structured HTTP requests. Each service accepts a set
of common parameters that define how the routing computation
should be performed.

Below is a summary of the primary parameters that are typically
included when interacting with OSRM endpoints.

### General Parameters

Most OSRM endpoints share a common structure. These parameters
control the behavior of the routing engine and the format of the
returned response.

| Parameter     | Description |
|--------------|-------------|
| **service**  | Specifies which OSRM service to invoke. Common values include `route`, `nearest`, `table`, `match`, `trip`, and `tile`. |
| **version**  | Indicates the API version of the OSRM service. For OSRM 5.x installations, this is typically `v1`. |
| **profile**  | Defines the transportation mode used for routing (e.g., `car`, `bike`, `foot`, or a custom Lua profile). This determines how the routing graph was generated. |
| **coordinates** | A sequence of coordinate pairs formatted as `{longitude},{latitude};{longitude},{latitude};...`. Encoded polylines may also be used. |
| **format**   | Specifies the response format. Currently, `json` is supported and used by default. |


### Constructing the Request

To interact with the OSRM routing engine, we define the
core configuration variables including the service type,
API version, routing profile, and host address.

These values are combined to dynamically construct
the endpoint URL used for sending requests.


In [ ]:
service = 'route'
version = 'v1'
profile = 'driving'
host = 'http://localhost:5000'

The route endpoint follows a structured pattern:

/route/{version}/{profile}/{coordinates}

Additional query parameters may be appended to refine
the response. These include:

- `alternatives` → return multiple route options
- `steps` → include turn-by-turn instructions
- `geometries` → define geometry encoding format
- `overview` → control the level of route summary detail
- `annotations` → include metadata such as distance and duration per segment

When requesting a route between multiple waypoints,
the order of coordinates is significant.

The routing engine calculates the path sequentially
based on the order provided. Therefore, to compute
a route from Point A → Point B → Point C,
the coordinates must be listed in that exact sequence.


In [ ]:
%pip install requests

Confirm your version_

In [ ]:
import requests
requests.__version__

In [ ]:
import requests

points = [point_1, point_2, point_3]

coordinates = ";".join(
    [f"{point.longitude},{point.latitude}" for point in points]
)

url = (
    f"{host}/{service}/{version}/{profile}/"
    f"{coordinates}?overview=full&steps=true&geometries=geojson"
)

response = requests.get(url)
response.status_code


The HTTP status code `200` confirms that the request was successfully
processed by the OSRM server.

With the connection validated, we can now parse the JSON response
and inspect the returned routing data.

Confirm your coordinates

In [ ]:
coordinates

In [ ]:
data = response.json()
data.keys()

The response body is returned in JSON format.  
We convert it into a Python dictionary to examine the structure
of the data and identify the available fields.

In [ ]:
routes = data['routes']
print(f"Found {len(routes)} route(s)")

The routing information is stored under the `routes` key in the response.

Since we did not request alternative paths, the response is expected
to contain a single route. We extract the routes list and verify
how many routes were returned.

Each route object contains detailed metadata describing the computed path.

From the selected route, we extract:

- `geometry.coordinates` → the sequence of longitude/latitude pairs
- `distance` → total travel distance (in meters)
- `duration` → estimated travel time (in seconds)

These values will later be used for visualization and analysis.

In [ ]:
route = routes[0]
route_coordinates = route['geometry']["coordinates"]
route_distance = route["distance"]  # unit: meters
route_duration = route["duration"]  # unit: seconds

route_coordinates[:5]

## Visualizing the Route

The geometry returned by OSRM is structured as a list of
coordinate pairs in the format:

[longitude, latitude]

However, Folium expects coordinates in the order:

[latitude, longitude]

To correctly render the route on the map, we must
reverse the order of each coordinate pair.

We iterate through the route geometry and swap
each coordinate pair so it matches the format
required by the Folium mapping engine.

In [ ]:
route_coordinates = [[point[1], point[0]] for point in route_coordinates]
route_coordinates[:5]

With the coordinates properly formatted, we now
initialize the base map and overlay the computed route.

A popup is attached to the route line to display
key metrics including total distance and estimated
travel duration.

In [ ]:
folium_map = get_folium_map(center_point, [point_1, point_2, point_3])

popup_text = (
    f"Distance: {route_distance} meters<br>"
    f"Duration: {route_duration} seconds"
)

popup = folium.Popup(popup_text, max_width=300)

folium.PolyLine(
    locations=route_coordinates,
    color="blue",
    weight=5,
    popup=popup
).add_to(folium_map)

folium_map

## Route Segmentation (Leg Analysis)

When multiple waypoints are provided in a routing request,
OSRM divides the overall route into segments known as *legs*.

Each leg represents the path between two consecutive waypoints.
For example:

- Leg 1 → from Point 1 to Point 2  
- Leg 2 → from Point 2 to Point 3  

This segmentation allows us to analyze intermediate portions
of the journey independently.

Every leg is further subdivided into *steps*.

A step describes a specific maneuver or road transition,
such as turning, merging, continuing straight, or arriving
at a waypoint.

By iterating through these steps, we can access the
geometry and metrics associated with each segment
of the route.

To better understand the route structure, we visualize
each leg separately on the map.

Distinct colors are assigned to differentiate between legs,
and popups are attached to display leg-specific metrics
such as distance and duration.

In [ ]:
folium_map = get_folium_map(center_point, [point_1, point_2, point_3])

colors = ['black', 'blue']

legs = data["routes"][0]["legs"]
for i, leg in enumerate(legs):
    steps = leg["steps"]
    for step in steps:
        geometry = step["geometry"]
        coordinates = [[point[1], point[0]]
                       for point in geometry["coordinates"]]
        color = colors[i % len(colors)]
        popup_text = f"""
        Leg number {i + 1}<br>
        Leg distance: {leg['distance']} meters<br>
        Leg duration: {leg['duration']} seconds<br>
        """
        popup = folium.Popup(popup_text, max_width=300)
        polyline = folium.PolyLine(
            locations=coordinates, color=color, weight=6, popup=popup)
        folium_map.add_child(polyline)

folium_map

## Step-Level Inspection

To gain deeper insight into the routing structure,
we isolate a single leg and analyze its internal steps.

Each step represents a discrete driving instruction,
capturing how the vehicle transitions from one road
segment to the next.

A step typically contains:

- The geometry of the road segment  
- The distance covered within that segment  
- The estimated duration  
- The road name (if available)  
- Maneuver information describing the type of movement  
- Intersection metadata  

By examining these elements, we can understand how
the route is constructed at a granular level.

For demonstration purposes, we focus only on
the first two steps of the selected leg.

This allows us to visualize how OSRM structures
individual maneuver segments and how they appear
when rendered independently on the map.

In [ ]:
folium_map = get_folium_map(
    point_1, [point_1, point_2, point_3], zoom_level=16)

colors = ['black', 'blue']

legs = data["routes"][0]["legs"]
for leg in legs:
    steps = leg["steps"]
    for i, step in enumerate(steps[:2]):
        geometry = step["geometry"]
        coordinates = [[point[1], point[0]]
                       for point in geometry["coordinates"]]
        color = colors[i % len(colors)]
        popup_text = f"""
        Distance: {step['distance']} meters
        <br>Duration: {step['duration']} seconds
        <br>Name: {step['name']}
        <br>Driving side: {step['driving_side']}
        <br>Instruction: {step['maneuver']['modifier']}
        <br>Type of maneuver: {step['maneuver']['type']}
        <br>Intersections: {len(step['intersections'])}
        """
        popup = folium.Popup(popup_text, max_width=300)
        polyline = folium.PolyLine(
            locations=coordinates, color=color, weight=6, popup=popup)
        folium_map.add_child(polyline)
    break

folium_map

To better understand the structure of a route segment,
we extract a single step from the first leg.

This allows us to examine the raw metadata returned
by the OSRM engine for one maneuver.

Each step contains detailed information describing
a specific road segment, including:

- `distance` → length of the segment (in meters)
- `duration` → estimated traversal time (in seconds)
- `name` → road name (if available)
- `driving_side` → side of the road used for traffic
- `geometry` → coordinate sequence of the segment
- `maneuver` → instruction type and direction
- `intersections` → intersection-level metadata

Printing the step dictionary provides a clear view
of how OSRM encodes maneuver-level routing data.

To make the step-level data more intuitive,
we render the segment on the map and attach
a popup containing the relevant attributes.

This allows us to visually connect the numerical
metadata with the actual geographic segment.

In [ ]:
from pprint import pprint

leg = data["routes"][0]["legs"][0]
step = leg["steps"][0]
pprint(step)

In addition to plotting the segment geometry,
we visualize the heading direction using the
`bearing` value provided in the maneuver object.

The bearing represents the compass direction
(in degrees) of travel at that specific point.

To illustrate this, we compute a short directional
line extending from the maneuver location,
providing a visual representation of the vehicle's heading.

In [ ]:
import math


def add_compass(folium_map: Map, location: tuple, bearing: int, size: int = 60):

    lat, lon = location
    end_lat = lat + (math.cos(math.radians(bearing)) * size * 1e-5)
    end_lon = lon + (math.sin(math.radians(bearing)) * size * 1e-5)

    folium.Marker(
        location=location,
        icon=folium.DivIcon(
            html="""<div style="font-size: 12px; color: red;">📍</div>"""),
        tooltip=f"Bearing: {bearing}°",
    ).add_to(folium_map)

    folium.PolyLine(
        locations=[location, (end_lat, end_lon)],
        color="red",
        weight=2,
    ).add_to(folium_map)


folium_map = get_folium_map(
    point_1, [point_1, point_2, point_3], zoom_level=16)

colors = ['black', 'blue']
legs = data["routes"][0]["legs"]
for leg in legs[:1]:
    steps = leg["steps"]
    for i, step in enumerate(steps[:1]):
        geometry = step["geometry"]
        coordinates = [[point[1], point[0]]
                       for point in geometry["coordinates"]]
        color = colors[i % len(colors)]

        popup_text = f"""
        Distance: {step['distance']} meters
        <br>Duration: {step['duration']} seconds
        <br>Name: {step['name']}
        <br>Driving side: {step['driving_side']}
        <br>Instruction: {step['maneuver']['modifier']}
        <br>Type of maneuver: {step['maneuver']['type']}
        <br>Intersections: {len(step['intersections'])}
        """
        popup = folium.Popup(popup_text, max_width=300)

        polyline = folium.PolyLine(
            locations=coordinates, color=color, weight=6, popup=popup
        )
        folium_map.add_child(polyline)

        maneuver_location = step["maneuver"]["location"]
        bearing = step["maneuver"].get("bearing_after", 0)
        add_compass(
            folium_map=folium_map,
            location=(maneuver_location[1], maneuver_location[0]),
            bearing=bearing
        )

folium_map

## Intersection-Level Analysis

Beyond steps and maneuvers, OSRM also provides
detailed information about intersections encountered
along the route.

Each step may contain one or more intersection objects,
describing the road configuration at that specific location.

An intersection object typically includes:

- `out` → Index of the outgoing bearing selected at the intersection  
- `in` → Index of the incoming bearing (road used to approach the intersection)  
- `entry` → Boolean array indicating which outgoing directions are accessible  
- `bearings` → List of compass bearings (in degrees) representing all possible road directions  
- `location` → Geographic coordinates of the intersection [longitude, latitude]

Together, these attributes describe both the geometry and
the navigational constraints at the intersection.

The `bearings` array represents all potential road directions
radiating from the intersection.

The `entry` array determines whether movement is allowed
for each corresponding bearing.

The `out` index identifies which bearing was selected
by the routing engine for the computed path.

To better understand the structure of intersection metadata,
we extract and print the intersection objects from the
first step of the first leg.

In [ ]:
from pprint import pprint

leg = data["routes"][0]["legs"][0]
step = leg["steps"][0]
intersections = step["intersections"]
pprint(intersections)

To make the intersection data more intuitive,
we visualize the available bearings on the map.

Each bearing is projected as a short directional line
originating from the intersection location,
providing a graphical representation of the
possible travel directions.

In [ ]:
import math


def add_compass(
    folium_map: Map,
    location: tuple,
    bearing: int,
    size: int = 50,
    color: str = "red",
    weight: int = 2
):
    lat, lon = location
    end_lat = lat + (math.cos(math.radians(bearing)) * size * 1e-5)
    end_lon = lon + (math.sin(math.radians(bearing)) * size * 1e-5)

    folium.Marker(
        location=location,
        icon=folium.DivIcon(
            html="""<div style="font-size: 12px; color: red;">📍</div>"""),
    ).add_to(folium_map)

    folium.PolyLine(
        locations=[location, (end_lat, end_lon)],
        color=color,
        weight=weight,
        tooltip=f"Bearing: {bearing}°",
    ).add_to(folium_map)


folium_map = get_folium_map(
    point_1, [point_1, point_2, point_3], zoom_level=18)

colors = ['black', 'blue']
legs = data["routes"][0]["legs"]
for leg in legs[:1]:
    steps = leg["steps"]
    for i, step in enumerate(steps[:1]):
        geometry = step["geometry"]
        coordinates = [[point[1], point[0]]
                       for point in geometry["coordinates"]]
        color = colors[i % len(colors)]

        popup_text = f"""
        Distance: {step['distance']} meters
        <br>Duration: {step['duration']} seconds
        <br>Name: {step['name']}
        <br>Driving side: {step['driving_side']}
        <br>Instruction: {step['maneuver']['modifier']}
        <br>Type of maneuver: {step['maneuver']['type']}
        <br>Intersections: {len(step['intersections'])}
        """
        popup = folium.Popup(popup_text, max_width=300)

        polyline = folium.PolyLine(
            locations=coordinates, color=color, weight=6, popup=popup
        )
        folium_map.add_child(polyline)

        intersections = step["intersections"]
        for intersection in intersections[:1]:
            location = intersection["location"]
            bearings = intersection["bearings"]
            for j, bearing in enumerate(bearings):
                allowed_for_traversal = intersection["entry"][j]
                add_compass(
                    folium_map=folium_map,
                    location=(location[1], location[0]),
                    bearing=bearing,
                    color="green" if allowed_for_traversal else "red",
                    weight=5
                )

folium_map

After progressing along the route, we encounter an
intersection that exposes three possible bearing values.

These bearings represent the potential travel directions
available at that location — such as continuing forward,
turning left, or reversing direction.

The `entry` array indicates which of these directions
are permitted. In this case, the bearing corresponding
to reversing direction is marked as `False`, meaning
that movement in that direction is not allowed.

The remaining bearings are marked as `True`,
indicating valid and accessible paths.

In [ ]:
import math
import folium
from folium import Map

def add_compass(
    folium_map: Map,
    location: tuple,
    bearing: int,
    size: int = 50,
    color: str = "green",
    weight: int = 5
):
    lat, lon = location
    end_lat = lat + (math.cos(math.radians(bearing)) * size * 1e-5)
    end_lon = lon + (math.sin(math.radians(bearing)) * size * 1e-5)

    # Arrow line (direction chosen by OSRM)
    folium.PolyLine(
        locations=[location, (end_lat, end_lon)],
        color=color,
        weight=weight,
        tooltip=f"Bearing: {bearing}°"
    ).add_to(folium_map)

# ---------------- BASE MAP ----------------
folium_map = get_folium_map(
    point_2, [point_1, point_2, point_3], zoom_level=20
)

# ✅ Point-2 marker (ADD ONLY ONCE)
folium.Marker(
    location=(point_2.latitude, point_2.longitude),
    popup="Point 2",
    icon=folium.Icon(color="blue", icon="info-sign")
).add_to(folium_map)

# ---------------- ROUTE LOGIC ----------------
legs = data["routes"][0]["legs"]

# leg[0] = point1 → point2
# leg[1] = point2 → point3   ✅ THIS IS WHAT YOU WANT
leg = legs[1]

steps = leg["steps"]

# Take FIRST STEP AFTER POINT-2
step = steps[0]

# Draw route geometry
geometry = step["geometry"]
coordinates = [[p[1], p[0]] for p in geometry["coordinates"]]

popup_text = f"""
Distance: {step['distance']} meters
<br>Duration: {step['duration']} seconds
<br>Name: {step['name']}
<br>Driving side: {step['driving_side']}
<br>Instruction: {step['maneuver']['modifier']}
<br>Type of maneuver: {step['maneuver']['type']}
"""

folium.PolyLine(
    locations=coordinates,
    color="black",
    weight=6,
    popup=folium.Popup(popup_text, max_width=300)
).add_to(folium_map)

# ---------------- INTERSECTION (KEY FIX) ----------------
for intersection in step["intersections"]:
    out_index = intersection.get("out")

    if out_index is None:
        continue  # no routing decision here

    bearing = intersection["bearings"][out_index]
    location = intersection["location"]

    add_compass(
        folium_map=folium_map,
        location=(location[1], location[0]),
        bearing=bearing
    )

folium_map

## Route Cost Analysis

In addition to distance and duration, OSRM returns a
`weight` value representing the routing cost.

The weight is calculated using the selected routing
profile's internal weighting algorithm. Depending on
the profile configuration, this value may correspond
to travel time, road priority, or other custom metrics.

For the default `car` profile, the weight often aligns
closely with travel duration. However, this relationship
is not mandatory and can vary if the profile's
Lua configuration modifies the weighting strategy.

Therefore, `distance`, `duration`, and `weight`
should be interpreted as distinct but related metrics.

In [ ]:
weight = route["weight"]
weight_name = route["weight_name"]
duration = route["duration"]
distance = route["distance"]

print(f"Weight: {weight} - {weight_name}")
print(f"Duration: {duration} seconds")
print(f"Distance: {distance} meters")

We extract the following route-level metrics:

- `weight` → computed routing cost  
- `weight_name` → identifier of the weighting strategy  
- `duration` → estimated travel time (seconds)  
- `distance` → total route length (meters)

## Alternative Route Generation

By default, OSRM returns a single optimal route.

To request additional route options, the
`alternatives` parameter must be enabled in the query.
This instructs the routing engine to compute
multiple feasible paths between the given waypoints.

In [ ]:
import requests

points = [point_1, point_2, point_3]
coordinates = ';'.join(
    [f'{point.longitude},{point.latitude}' for point in points])

url = f"{host}/{service}/{version}/{profile}/{coordinates}?alternatives=3&overview=full&steps=true&geometries=geojson"

response = requests.get(url)
data = response.json()
routes = data['routes']
print(f"Found {len(routes)} route(s)")

Even when the `alternatives` parameter is set to a
value greater than one, the final number of routes
depends entirely on the topology of the road network
and the routing constraints defined by the profile.

In this case, only a single route was available
despite requesting multiple alternatives.

## Conclusion

This notebook demonstrated how to interact with
the OSRM routing engine programmatically using Python.

We explored:

- Route generation between multiple waypoints  
- Leg and step-level route decomposition  
- Intersection metadata and directional bearings  
- Route cost metrics including weight, duration, and distance  
- Alternative route requests  

Together, these components illustrate how OSRM
encodes routing logic and how it can be visualized
and analyzed for navigation and geospatial applications.